In [1]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [3]:
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00001939').resource

In [4]:
#memunculkan study_name
std

<ResourceObject: studies: MGYS00001939 (2053604286024) (2053604286024)>

In [5]:
#melihat jumlah sample pada study di atas
std.samples_count

12

In [6]:
#memunculkan list sample sesuai jumlah
std.samples

[<ResourceObject: samples: ERS1851088 (2053597982168) (2053597982168)>,
 <ResourceObject: samples: ERS1851084 (2053604912152) (2053604912152)>,
 <ResourceObject: samples: ERS1851093 (2053605028416) (2053605028416)>,
 <ResourceObject: samples: ERS1851087 (2053605029424) (2053605029424)>,
 <ResourceObject: samples: ERS1851094 (2053605030432) (2053605030432)>,
 <ResourceObject: samples: ERS1851086 (2053605031440) (2053605031440)>,
 <ResourceObject: samples: ERS1851095 (2053605077232) (2053605077232)>,
 <ResourceObject: samples: ERS1851090 (2053605078240) (2053605078240)>,
 <ResourceObject: samples: ERS1851092 (2053605079248) (2053605079248)>,
 <ResourceObject: samples: ERS1851085 (2053605080256) (2053605080256)>,
 <ResourceObject: samples: ERS1851091 (2053605110000) (2053605110000)>,
 <ResourceObject: samples: ERS1851089 (2053605111008) (2053605111008)>]

In [7]:
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

[{'key': 'investigation type', 'unit': None, 'value': 'metatranscriptome'},
 {'key': 'geographic location (longitude)', 'unit': None, 'value': '5.411394'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'Belgium'},
 {'key': 'collection date', 'unit': None, 'value': '2015-03-18'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'area of cropland'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'area of cropland'},
 {'key': 'environment (material)', 'unit': None, 'value': 'agricultural soil'},
 {'key': 'environmental package', 'unit': None, 'value': 'soil'},
 {'key': 'sequencing method', 'unit': None, 'value': 'Illumina'},
 {'key': 'elevation', 'unit': None, 'value': '46.0'},
 {'key': 'geographic location (latitude)', 'unit': None, 'value': '50.93487'},
 {'key': 'instrument model', 'unit': None, 'value': 'Illumina HiSeq 4000'}]

In [ ]:
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [18]:
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00001939').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
sample = std.samples

MGYS00001939


In [20]:
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00135103
analyses: MGYA00135102
analyses: MGYA00135099
analyses: MGYA00135098
analyses: MGYA00135093
analyses: MGYA00135104
analyses: MGYA00135094
analyses: MGYA00135095
analyses: MGYA00135096
analyses: MGYA00135100
analyses: MGYA00135101
analyses: MGYA00135097


In [27]:
for x in sample:
    print('sample: '+str(x.accession))
    #for a in x.runs:
        #print('runs: '+str(a.accession))
        #for b in a.analyses:
            #print('analyses: '+str(b.accession))

sample: ERS1851088
sample: ERS1851084
sample: ERS1851093
sample: ERS1851087
sample: ERS1851094
sample: ERS1851086
sample: ERS1851095
sample: ERS1851090
sample: ERS1851092
sample: ERS1851085
sample: ERS1851091
sample: ERS1851089


In [21]:
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')

In [22]:
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
merged

,ID_Sample,Sample Name,ID_Analyses,investigation type,geographic location (longitude),"geographic location (country and/or sea,region)",collection date,environment (biome),environment (feature),environment (material),environmental package,sequencing method,elevation,geographic location (latitude),instrument model
0,ERS1851088,"Bulk soil exposed to nanosilver, sample 2",MGYA00135103,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
1,ERS1851084,"Bulk soil not exposed to nanosilver, sample 1",MGYA00135102,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
2,ERS1851093,"Rhizosphere soil exposed to nanosilver, sample 1",MGYA00135099,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
3,ERS1851087,"Bulk soil exposed to nanosilver, sample 1",MGYA00135098,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
4,ERS1851094,"Rhizosphere soil exposed to nanosilver, sample 2",MGYA00135093,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
5,ERS1851086,"Bulk soil not exposed to nanosilver, sample 3",MGYA00135104,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
6,ERS1851095,"Rhizosphere soil exposed to nanosilver, sample 3",MGYA00135094,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
7,ERS1851090,"Rhizosphere soil not exposed to nanosilver, sa...",MGYA00135095,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
8,ERS1851092,"Rhizosphere soil not exposed to nanosilver, sa...",MGYA00135096,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000
9,ERS1851085,"Bulk soil not exposed to nanosilver, sample 2",MGYA00135100,metatranscriptome,5.411394,Belgium,2015-03-18,area of cropland,area of cropland,agricultural soil,soil,Illumina,46.0,50.93487,Illumina HiSeq 4000


In [24]:
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

D:\Seminar\Seminar_Anam\Study_Jagung


In [25]:
#masuk ke folder "Study_Jagung"
%cd Study_Jagung
#export ke .csv
merged.to_csv(std.accession+'.csv')

D:\Seminar\Seminar_Anam\Study_Jagung


In [9]:
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

analyses: MGYA00135103
analyses: MGYA00135102
analyses: MGYA00135099
analyses: MGYA00135098
analyses: MGYA00135093
analyses: MGYA00135104
analyses: MGYA00135094
analyses: MGYA00135095
analyses: MGYA00135096
analyses: MGYA00135100
analyses: MGYA00135101
analyses: MGYA00135097


In [ ]:
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
blank = []

In [ ]:
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
filename_sample = '' 

In [ ]:
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [13]:
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()

D:\Seminar\Seminar_Anam\Study_Jagung
